In [ ]:
import pandas as pd

In [ ]:
import os
import pickle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

#Q1
#Calculate the number of columns
print(len(df.columns))
print(df.columns)

In [ ]:
#Q2
#Calculate the Standard Deviation if the trips duration
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

# Calculate the standard deviation without filtering
std_duration_no_filter = df.duration.std()

print(f"{std_duration_no_filter:.2f}")

In [ ]:
print(len(df.columns))
print(df.columns)

In [ ]:
#Q3
# Calculate duration in minutes
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60

# Total number of records before filtering
n_total = len(df)

# Filter for durations between 1 and 60 minutes (inclusive)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Number of records after filtering
n_filtered = len(df_filtered)

# Fraction left in percentage
fraction = (n_filtered / n_total) * 100

print(f"{fraction:.0f}")

In [ ]:
# Only use location IDs
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical + numerical].to_dict(orient='records')

# One-hot encode
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# 5. Get the dimensionality (number of columns)
dimensionality = X_train.shape[1]
print(f"Dimensionality of the feature matrix: {dimensionality}")

In [ ]:
# Train model
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print(f"RMSE: {rmse}")

In [ ]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
len(df), len(df_val)

In [ ]:
df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)